In [1]:
%%capture
!pip install biosppy

In [ ]:
!gdown 1f0vsok44_ZwKnLK2AGRqYyw0FmYcxrCT #test.zip
!gdown 1zXWuODLYE3UFgdUqn4HCnF93Ubg48sAm #sample_submissions к открытому тесту

!gdown 1-mmBzNTmOZPeJ0GaLq6C6P6y_bZGRg7u #eff_b0 val 0.995
!gdown 1-VmDYlQkGxjuF0SQEnVnhT2Sj1j6l0VI #convnext val 0.988
!gdown 1-PuuH25Qvg9uHFX2uc1Y0CddkGR5-atd #regnet_3gf val 0.989
!gdown 1-O2RNyOhA-do7Jroq5OJE_lNMDyv2CCi #eff_v2_s val 0.994

!unzip test.zip

In [3]:
import pandas as pd
import numpy as np
from tqdm.auto import tqdm
import os

import torch
from torchvision import models
from scipy.signal import savgol_filter
import biosppy

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device

device(type='cuda')

###**Подгрузка моделей**
В каждой модели архитектура изменена на вход с одним каналом, и добавлен линейный слой на 2 выхода

In [4]:
%%capture
"""
Модель: EfficientnetV2
Размер: 80МБ
Кол-во параметров: 21 458 488
"""

efficientnet_v2_s = models.efficientnet_v2_s(pretrained=True)

efficientnet_v2_s.classifier[1] = torch.nn.Linear(in_features=efficientnet_v2_s.classifier[1].in_features, out_features=2, bias=True)
efficientnet_v2_s.features[0][0] = torch.nn.Conv2d(1, 24, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)

efficientnet_v2_s.load_state_dict(torch.load('/content/eff_v2_s_R_peaks_model_f1:0.994_6500.pth', map_location=torch.device('cuda')))

efficientnet_v2_s.to(device)
efficientnet_v2_s.eval()
pass

In [5]:
%%capture
"""
Модель: Convnext Tiny
Размер: 100МБ
Кол-во параметров: 	28 589 128
"""

convnext_tiny = models.convnext_tiny(pretrained=True)

convnext_tiny.features[0][0] = torch.nn.Conv2d(1, 96, kernel_size=(4, 4), stride=(4, 4), padding=(2, 2))

convnext_tiny.classifier[2] = torch.nn.Linear(in_features=768, out_features=2, bias=True)

"""
Изначально в модели свертки настроены для 3 каналов и из-за их последовательности, 1 канал впревращается в тензор размера 0,
Делее идет изменение размера сверток для избежания ошибок размерности
"""
for seq in [2, 4, 6]:
    convnext_tiny.features[seq][1] = torch.nn.Conv2d(convnext_tiny.features[seq][1].in_channels, convnext_tiny.features[seq][1].out_channels, kernel_size=(2, 2), padding=(1, 1), stride=(2, 2))

convnext_tiny.load_state_dict(torch.load('/content/convnext_tiny_R_peaks_model_f1:0.988_8500.pth', map_location=torch.device('cuda')))

convnext_tiny.to(device)
convnext_tiny.eval()
pass

In [6]:
%%capture
"""
Модель: RegNet y 3 2gf
Размер: 74МБ
Кол-во параметров: 	19 436 338
"""

regnet_y_3_2gf = models.regnet.regnet_y_3_2gf(pretrained=True)

regnet_y_3_2gf.stem[0] = torch.nn.Conv2d(1, 32, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)

regnet_y_3_2gf.fc = torch.nn.Sequential(
    torch.nn.Dropout(p=0.1, inplace=True),
    torch.nn.ReLU(),
    torch.nn.Linear(in_features=regnet_y_3_2gf.fc.in_features, out_features=2, bias=True),
)

regnet_y_3_2gf.load_state_dict(torch.load('/content/regnet_y_3_2gf_R_peaks_model_f1:0.989_8500.pth', map_location=torch.device('cuda')))

regnet_y_3_2gf.to(device)
regnet_y_3_2gf.eval()
pass

In [7]:
%%capture
"""
Модель: Efficientnet B0
Размер: 20МБ
Кол-во параметров: 	5 288 548
"""

efficientnet_b0 = models.efficientnet.efficientnet_b0(pretrained=True)

efficientnet_b0.features[0][0] = torch.nn.Conv2d(1, 32, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
efficientnet_b0.classifier[1] = torch.nn.Linear(in_features=efficientnet_b0.classifier[1].in_features, out_features=2, bias=True)

efficientnet_b0.load_state_dict(torch.load('/content/eff_b0_R_peaks_model_f1:0.995_4500.pth', map_location=torch.device('cuda')))

efficientnet_b0.to(device)
efficientnet_b0 = efficientnet_b0.eval()
pass

### **Получение предсказаний**

In [14]:
def get_predicts(data_path, sample_submission=None):
    """
    data_path: путь к директроии с сигналами
    sample_submission: путь датафрейму с колонками record_name myocard который будет заполняться, если None создается на основе data_path
    """

    predict = pd.DataFrame(columns=['record_name', 'myocard'])

    # создание датафрейма для предсказаний
    if sample_submission is None:
        predict.record_name = [record[:-4] for record in os.listdir(data_path) if record[-4:] == '.npy']
    else:
        test = pd.read_csv(sample_submission)
        predict.record_name = test.record_name

    for name in tqdm(predict.record_name):
        data = np.load(f'{data_path}/{name}.npy')
        # Удаление шума
        data = np.apply_along_axis(lambda x: savgol_filter(x, 30, 2), axis=1, arr=data)

        # Получение индексов R пиков, если не найдены в одном канале, помск по остальным
        peaks = biosppy.signals.ecg.christov_segmenter(signal=data[10], sampling_rate = 500)[0]
        for channel in range(12):
            if len(peaks) > 3:
                break
            peaks = biosppy.signals.ecg.christov_segmenter(signal=data[channel], sampling_rate = 500)[0]

        # Создание каждого сингнала с окружностью 200 точек
        signals = [torch.tensor(data[:, i-200:i+200]).to(device) for i in peaks[1:-1]]

        signals = torch.stack(signals)[:, None, :, :]

        # Получение выходов всех моделей
        eff_logits = torch.nn.functional.softmax(efficientnet_b0(signals), dim=1)
        convnext_logits = torch.nn.functional.softmax(convnext_tiny(signals), dim=1)
        regnet_logits = torch.nn.functional.softmax(regnet_y_3_2gf(signals), dim=1)
        eff_v2_s_logits = torch.nn.functional.softmax(efficientnet_v2_s(signals), dim=1)

        peaks_logits = (eff_logits + convnext_logits + regnet_logits + eff_v2_s_logits) / 4
        peaks_logits = torch.argmax(peaks_logits, dim=1)

        # Берем то число которого больше среди предсказаний моделей
        ans = int(sum(peaks_logits) / len(peaks_logits) > 0.5)


        predict.loc[predict.record_name == name, 'myocard'] = ans

    return predict

In [15]:
predict = get_predicts(data_path='test')

  0%|          | 0/449 [00:00<?, ?it/s]

In [16]:
predict.groupby('myocard').count()

,record_name
myocard,
0,349
1,100


In [10]:
predict.to_csv('predict.csv', index=False)